## 1. Merge Root Files

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT'
starts_with = 'root'
output_name = 'merge'

RadLib.Merge_Roots(directory, starts_with, output_name)

## 2. Root to Dataframe

In [1]:
import Radiography_Library as RadLib

directory = 'RESULTS/'
root_name_starts = "RealTube"

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"

x_data, y_data = RadLib.Root_to_Dask(directory, root_name_starts, tree_name, x_branch, y_branch)

## 3. Dask Dataframe to Heatmap

In [ ]:
import Radiography_Library as RadLib

size = 70
log_factor = 1e-10

x_shift = 0
y_shift = 0

save_as = ''
htmp_array, xlim, ylim = RadLib.Heatmap_from_Dask(x_data, y_data, size, log_factor, x_shift, y_shift, save_as)

## 4.1. Isolate Bone from Tissue

Load Roots

In [ ]:
import Radiography_Library as RadLib

directory = 'RESULTS/'
# rootnames = ["RC40K100M", "RC80K100M"]
rootnames = ["prueba_40k", "prueba_80k"]

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"

x_1, y_1, x_2, y_2 = RadLib.LoadRoots(directory, rootnames, tree_name, x_branch, y_branch)

Calculate Heatmaps

In [ ]:
import matplotlib.pyplot as plt

x_shift = 0.0
y_shift = 0

size = 60
log_factor = 0.01

low_energy_img, xlim, ylim = RadLib.Heatmap_from_Dask(x_1, y_1, size, log_factor, x_shift, y_shift, '')
high_energy_img, xlim, ylim = RadLib.Heatmap_from_Dask(x_2, y_2, size, log_factor, x_shift, y_shift, '')

# low_energy_img  = (low_energy_img * 255).astype(np.uint8)
# high_energy_img = (high_energy_img * 255).astype(np.uint8)

plt.close()
plt.close()

Plot Heatmaps

In [ ]:
import Radiography_Library as RadLib

save_as = ''
SSH_Bone, ACNR_Bone = RadLib.IsolateTissues(low_energy_img, high_energy_img, save_as)

## 4.2. Bone Mineral Density (BMD)

In [ ]:
import Radiography_Library as RadLib
thickness_bone = RadLib.BMO(SLS_Bone, SLS_Tissue)

Plot Interactive Heatmap

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure(go.Heatmap(z = thickness_bone, x = xlim, y = ylim))
fig.update_layout(width = 400, height = 400, xaxis = dict(autorange = 'reversed'), yaxis = dict(autorange = 'reversed'))    
fig.show()

## 5.1. Calculate Interactive CNR

Trim Image

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

directory = 'RESULTS/'
image = 'a' + '.png'
image = Image.open(directory + image)
image = image.convert('L')

# cropped_image = image.crop((1000, 1300, image.width - 1000, image.width - 1300))
cropped_image = image
plt.imshow(cropped_image, cmap='gray')

Launch Interactive CNR

In [ ]:
%matplotlib widget
%matplotlib tk
import Radiography_Library as RadLib

RadLib.Interactive_CNR(cropped_image)

## 5.2. Calculate Fixed CNR 

In [ ]:
import Radiography_Library as RadLib

image_path = "RESULTS/" + "a" + ".png"
save_as = ''

shftx_s = 0.0 # shift x-coordinate signal box
shfty_s = 0.0 
shftx_b = 200.0 # shift x-coordinate background box
shfty_b = 0.0

coords_signal  = [1200 + shftx_s, 1000 + shfty_s, 1800 + shftx_s, 1800 + shfty_s] # x1, y1, x2, y2
coords_bckgrnd = [2100 + shftx_b, 1000 + shfty_b, 2300 + shftx_b, 1800 + shfty_b] # x1, y1, x2, y2

RadLib.Fixed_CNR(image_path, save_as, coords_signal, coords_bckgrnd)

## 6.1. Denoise with Skimage.Denoise_Bilateral

In [ ]:
import Radiography_Library as RadLib

# load array
path = SSH_Bone
isArray = True

# or load image
if isArray == False:
    directory = 'RESULTS/'
    path = directory + 'a' + '.png'

sigma_color = 0.05
sigma_spatial = 20

Denoised_Image = RadLib.Denoise_EdgeDetection(path, isArray)

## 6.2. Denoise by Fourier Transform

In [ ]:
import Radiography_Library as RadLib

array = htmp_array
isHann = False

alpha = 1

save_as = ''
isCrossSection = False # plot crosss-section

fft_image = RadLib.Denoise(array, isHann, alpha, save_as, isCrossSection)

## 7. Save Plotly Heatmap

In [ ]:
import Radiography_Library as RadLib

# array = htmp_array
array = Denoised_Image
# array = SLS_Bone

title   = r"$ \large{ \text{Denoised Isolated Bone, DEXA: 40, 80 keV, 250M Beams} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

annotation = 'CNR = 9.1' 

width  = 800
height = 800

save_as = ''

RadLib.Plotly_Heatmap(array, xlim, ylim, title, x_label, y_label, annotation, width, height, save_as)